In [1]:
from models.autoregressive import AutoregressiveModel
from models.xgboost import XGBoostModel

import pandas as pd

/opt/homebrew/Caskroom/miniforge/base/envs/test_bike/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
base_path = "../"
data = pd.read_parquet(base_path + "data/train.parquet")

## Autoregressive Model

In [6]:
time = "datetime"
space = "counter_name"
endog = "log_bike_count"

ar = AutoregressiveModel(time, "H", space, endog)
mod_data = ar.preprocess(data, base_path=base_path)

end_train = '2021-08-09 23:00:00'
train_data = mod_data[mod_data["datetime"] <= end_train]
test_data = mod_data[mod_data["datetime"] > end_train]

/Users/joaosilva/Current/Career/Projects/bike_counters/src/features/preprocessing.py:28: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  data.groupby(["counter_name", "date_truncated"])["log_bike_count"]
/Users/joaosilva/Current/Career/Projects/bike_counters/src/features/covid_features.py:64: DtypeWarning: Columns (2,3,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  covid_index = pd.read_csv(path)


In [4]:
ar.fit(train_data.drop(columns="log_bike_count"), train_data["log_bike_count"])

/opt/homebrew/Caskroom/miniforge/base/envs/test_bike/lib/python3.12/site-packages/skforecast/preprocessing/preprocessing.py:418: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  original_sizes = data.groupby(series_id).size()
/opt/homebrew/Caskroom/miniforge/base/envs/test_bike/lib/python3.12/site-packages/skforecast/preprocessing/preprocessing.py:420: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for k, v in data.groupby(series_id):
/opt/homebrew/Caskroom/miniforge/base/envs/test_bike/lib/python3.12/site-packages/skforecast/preprocessing/preprocessing.py:421: FutureWarning: 'H' is deprecated and will be removed

In [5]:
predictions = ar.predict(test_data.drop(columns="log_bike_count"))

/opt/homebrew/Caskroom/miniforge/base/envs/test_bike/lib/python3.12/site-packages/skforecast/preprocessing/preprocessing.py:478: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  original_sizes = data.groupby(series_id).size()
/opt/homebrew/Caskroom/miniforge/base/envs/test_bike/lib/python3.12/site-packages/skforecast/preprocessing/preprocessing.py:479: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  exog_dict = dict(tuple(data.groupby(series_id)))
/opt/homebrew/Caskroom/miniforge/base/envs/test_bike/lib/python3.12/site-packages/skforecast/preprocessing/preprocessing.py:481: FutureWarning: 'H' is deprecated and wil

In [6]:
merged_df = test_data.merge(predictions, on=['counter_name', 'datetime'], how='left')

In [7]:
from sklearn.metrics import root_mean_squared_error
root_mean_squared_error(merged_df["log_bike_count_x"], merged_df["log_bike_count_y"])

0.4908631309883157

## XGBoost

In [3]:
xgboost = XGBoostModel()
mod_data = xgboost.preprocess(data, base_path)

end_train = '2021-08-09 23:00:00'
train_data = mod_data[mod_data["datetime"] <= end_train]
test_data = mod_data[mod_data["datetime"] > end_train]

counter_id                         category
counter_name                       category
site_id                               int64
site_name                          category
bike_count                          float64
date                         datetime64[us]
counter_installation_date    datetime64[us]
coordinates                        category
counter_technical_id               category
latitude                            float64
longitude                           float64
log_bike_count                      float64
dtype: object


/Users/joaosilva/Current/Career/Projects/bike_counters/src/features/preprocessing.py:28: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  data.groupby(["counter_name", "date_truncated"])["log_bike_count"]
/Users/joaosilva/Current/Career/Projects/bike_counters/src/features/covid_features.py:64: DtypeWarning: Columns (2,3,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  covid_index = pd.read_csv(path)


In [4]:
X = train_data.drop(columns=["log_bike_count", 'datetime', 'counter_name'])
y = train_data["log_bike_count"]
xgboost.fit(X=X, y=y)

X_test = test_data.drop(columns=["log_bike_count", 'datetime', 'counter_name'])
predictions = xgboost.predict(X_test)
predictions = pd.concat([pd.Series(predictions, name="log_bike_count"), 
                         test_data[['counter_name', 'datetime']].reset_index()], axis=1)

In [5]:
predictions

,log_bike_count,index,counter_name,datetime
0,0.702969,449955,152 boulevard du Montparnasse O-E,2021-08-10 00:00:00
1,1.386959,449956,Pont des Invalides N-S,2021-08-10 00:00:00
2,1.104312,449957,152 boulevard du Montparnasse E-O,2021-08-10 00:00:00
3,0.031427,449958,Voie Georges Pompidou NE-SO,2021-08-10 00:00:00
4,1.102487,449959,18 quai de l'Hôtel de Ville NO-SE,2021-08-10 00:00:00
...,...,...,...,...
41659,2.555764,491614,18 quai de l'Hôtel de Ville SE-NO,2021-09-09 23:00:00
41660,3.107188,491615,Totem Cours la Reine E-O,2021-09-09 23:00:00
41661,3.161846,491616,38 rue Turbigo NE-SO,2021-09-09 23:00:00
41662,3.771682,491617,Totem 64 Rue de Rivoli E-O,2021-09-09 23:00:00


In [6]:
merged_df = test_data.merge(predictions, on=['counter_name', 'datetime'], how='left')

In [7]:
merged_df

,bike_count,log_bike_count_x,datetime,is_weekend,IsHoliday,StringencyIndex_Average,t,rr1,u,ht_neige,...,day_31,day_of_week_0,day_of_week_1,day_of_week_2,day_of_week_3,day_of_week_4,day_of_week_5,day_of_week_6,log_bike_count_y,index
0,1.0,0.693147,2021-08-10 00:00:00,0.0,0.0,47.07,290.35,0.0,80.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.702969,449955
1,3.0,1.386294,2021-08-10 00:00:00,0.0,0.0,47.07,290.35,0.0,80.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.386959,449956
2,2.0,1.098612,2021-08-10 00:00:00,0.0,0.0,47.07,290.35,0.0,80.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.104312,449957
3,0.0,0.000000,2021-08-10 00:00:00,0.0,0.0,47.07,290.35,0.0,80.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.031427,449958
4,2.0,1.098612,2021-08-10 00:00:00,0.0,0.0,47.07,290.35,0.0,80.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.102487,449959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41659,12.0,2.564949,2021-09-09 23:00:00,0.0,0.0,43.91,292.25,0.0,91.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.555764,491614
41660,22.0,3.135494,2021-09-09 23:00:00,0.0,0.0,43.91,292.25,0.0,91.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.107188,491615
41661,23.0,3.178054,2021-09-09 23:00:00,0.0,0.0,43.91,292.25,0.0,91.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.161846,491616
41662,45.0,3.828641,2021-09-09 23:00:00,0.0,0.0,43.91,292.25,0.0,91.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.771682,491617


In [8]:
from sklearn.metrics import root_mean_squared_error
root_mean_squared_error(merged_df["log_bike_count_x"], merged_df["log_bike_count_y"])

0.032272291421259654